<a href="https://colab.research.google.com/github/chechelan/0-chechelan/blob/main/rayminer_random_forest_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
import joblib

# Load the previously saved model
clf_best = joblib.load('random_forest_model.pkl')

api_key = "26NKG48ALQA5LNWCUCW6FFLTU"

def get_hourly_weather_data(location, date_range, api_key):
    all_data = []
    url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{location}/{date_range}?unitGroup=metric&include=hours&key={api_key}&contentType=json"
    response = requests.get(url)

    if response.status_code != 200:
        print(f'Unexpected status code for {location}:', response.status_code)
        return None

    jsonData = response.json()

    # Extract hourly data
    for day in jsonData['days']:
        for hour in day['hours']:
            all_data.append({
                'city': location,
                'date': day['datetime'],
                'datetime': hour['datetime'],
                'uvindex': hour['uvindex'],
                'temp': hour['temp'],
                'conditions': hour['conditions'],
                'icon': hour['icon'],
                'cloudcover': hour['cloudcover']
            })

    return pd.DataFrame(all_data)

def predict_reminder(api_key):
    # 1. User Input
    age = int(input("Enter your age: "))
    gender = input("Enter your gender (male/female): ")
    skintype = int(input("Enter your skin type (1-6): "))
    makeup = input("Do you wear makeup (yes/no): ")
    where = input("Where are you (indoors/outdoors): ")
    SPF = int(input("Enter the SPF value of your sunscreen: "))
    forwhom = input("Is this sunscreen for yourself or a child (myself/child): ")
    location = input("Enter your location (city, country): ")
    hour_of_interest = input("Enter the hour of interest (e.g. '14:00'): ")

    # 2. Fetch Hourly Weather Data
    today = pd.Timestamp.now().strftime('%Y-%m-%d')
    hourly_data = get_hourly_weather_data(location, today, api_key)
    hour_data = hourly_data[hourly_data['datetime'] == f"{today}T{hour_of_interest}:00"]

    # Ensure we have data for the specified hour
    if hour_data.empty:
        print(f"No data available for {hour_of_interest} hour.")
        return None

    # Add user input to the data
    hour_data['age'] = age
    hour_data['gender'] = gender
    hour_data['skintype'] = skintype
    hour_data['makeup'] = makeup
    hour_data['where'] = where
    hour_data['SPF'] = SPF
    hour_data['forwhom'] = forwhom

    # 3. Preprocess Data and Predict
    processed_data = preprocess_new_data(hour_data, label_encoders)
    prediction = clf_best.predict(processed_data)

    return prediction[0]

print("You should set a reminder:", predict_reminder(api_key))